In [1]:
%load_ext wurlitzer
import pickle
import trimesh as tm
import pyrender
import numpy as np
import visibility
from importlib import reload
reload(visibility)
from visibility.visibility import Visibility
from joblib import Parallel,delayed
from tqdm import tqdm
from scipy.sparse import lil_matrix
from klampt import WorldModel,Geometry3D
from klampt import vis
from klampt.math import so3,se3
import trimesh
import os
import random
import numpy as np
import pdb
import math
from klampt.model.create import moving_base_robot
from klampt.io import resource
from klampt.math import vectorops,so3
from klampt.model.collide import WorldCollider
from klampt.model import ik
import time
import numpy as np
from tqdm import tqdm
from klampt import IKObjective
import networkx as nx
from klampt.model.trajectory import RobotTrajectory,Trajectory
from klampt.model.create import *
from klampt import vis
from klampt.robotsim import Geometry3D,WorldModel
from klampt.math import se3,so3,vectorops
from klampt.vis import colorize
import sys
import time
from klampt.model import trajectory
from threading import Thread,Lock,Event



mesh_file = './full_detail_hospital_cad_meters_fixed.obj'
pcd_file = "towerbot_robot_used_points_hospital.pcd"
reachable_points_file = 'towerbot_reachable.p'
irradiance_file = 'towerbot_irradiance_matrix_35_divs.p'
sampling_places_file = 'towerbot_sampling_places_35_divs.p'
configs_file = 'configs.p'
robot_file = "./primitives/towerbot.rob"
solutions_file = 'towerbot_solutions.p'
adjacency_file = 'towerbot_adj.p'
roadmap_file = 'towerbot_roadmap.p'
node_coords_file = 'towerbot_node_coords.p'
movie_dir = '../media/movies/towerbot/30_min/{:06d}.png'
tour_file = 'Towerbot_currTSP.txt'
skip = 1
dt = 10
max_step = 20
scale = 1
linknum = 2

def generate_video(mesh_file = './remeshed_hospital_room_full_35k.obj',
                   robot_file = "./data/towerbot.rob",
                   movie_dir = './media/movies/towerbot/30_min',
                   tour_file = 'towerbot_currTSP.txt',
                   results_dir = './3D_results',
                   experiment = '30_min',
                   resolution = 250,
                   skip = 1,
                   dt = 0.4,
                   max_step = 20,
                   scale = 4,
                   linknum = 2,
                   robot_name =  'towerbot'):

    mesh_name = mesh_file.split('.')[-2]
    results_dir = results_dir+'/{}'.format(mesh_name)
    if(not os.path.exists(results_dir)):
        os.mkdir(results_dir)

    results_dir = results_dir + '/{}'.format(experiment)
    if(not os.path.exists(results_dir)):
        os.mkdir(results_dir)

    results_dir = results_dir + '/{}'.format(robot_name)
    if(not os.path.exists(results_dir)):
        os.mkdir(results_dir)


    pcd_file = results_dir + "/{}_used_points_{}_divs.pcd".format(robot_name,resolution)
    reachable_points_file = results_dir + '/{}_reachable_{}_divs.p'.format(robot_name,resolution)
    irradiance_file = results_dir + '/{}_irradiance_matrix_{}_divs.p'.format(robot_name,resolution)
    sampling_places_file = results_dir + '/{}_sampling_places_{}_divs.p'.format(robot_name,resolution)
    configs_file = results_dir + '/{}_configs_{}_divs.p'.format(robot_name,resolution)
    solutions_file = results_dir + '/{}_solutions_{}_divs.p'.format(robot_name,resolution)
    adjacency_file = results_dir + '/{}_adjacency_{}_divs.p'.format(robot_name,resolution)
    roadmap_file = results_dir + '/{}_roadmap_{}_divs.p'.format(robot_name,resolution)
    node_coords_file = results_dir + '/{}_node_coords_{}_divs.p'.format(robot_name,resolution)

    irradiance_matrix = pickle.load(open(irradiance_file,'rb'))
    reachable = pickle.load(open(reachable_points_file,'rb'))
    configs = pickle.load(open(configs_file,'rb'))
    sampling_places = pickle.load(open(sampling_places_file,'rb'))

    #creating movie directory
    os.makedirs(movie_dir, exist_ok=True)
    # fixing movie paths
    movie_folder = movie_dir
    movie_dir += '/{:06d}.png'
    
    sampling_places = sampling_places[reachable]
    configs = np.array(configs)[reachable].tolist()
    mu_single = 60*60*80*0.5*irradiance_matrix
    print("\n\n number of reachable points : {} \n".format(np.sum(reachable)))




    #process irradiation spots
    irradiance_matrix = pickle.load(open(irradiance_file,'rb'))

    vis_tester =Visibility(mesh_file,res = 512, useShader = True,createWnd = True)
    m = vis_tester.mesh
#     m=Mesh(mesh_file)
#     areas = m.area()
    areas = m.area()
    
    
    areas = m.area()
    visible_points = np.asarray(((irradiance_matrix.sum(axis = 0)) > 0)).flatten()
    visible_areas = areas[visible_points]

    total_visible_areas = visible_areas.sum()
    total_visible_areas


#     irradiance_matrix = irradiance_matrix[reachable]

    visible_points = np.asarray(((irradiance_matrix.sum(axis = 0)) > 0)).flatten()

    visible_irradiance = irradiance_matrix[:,visible_points]


    visible_areas = areas[visible_points]
    # we then define the visible area weights:



    visible_area_weights = np.exp(100*(visible_areas/visible_areas.sum()))
    visible_area_weights[:] = 1


    solutions = pickle.load(open(solutions_file,'rb'))
    visible_points = np.asarray(((irradiance_matrix.sum(axis = 0)) > 0)).flatten()
    visible_irradiance = irradiance_matrix[:,visible_points]



    # we now calculate the euclidean distance matrices for all the non-zero points:
    times = solutions[0]
    total_time = times.sum()
    points_mask = times>0.5
    used_points = sampling_places[points_mask,:]
    print(used_points.shape)
    mus = 80*np.array(np.matmul(visible_irradiance.todense().transpose(),times)).flatten()
    coverage = visible_areas[mus > 280].sum()/243.6
    print('Total Environment Coverage = {:.2f}%'.format(100*coverage))

    adjacency_matrix = pickle.load(open(adjacency_file,'rb'))
    roadmap = pickle.load(open(roadmap_file,'rb'))
    node_coords = pickle.load(open(node_coords_file,'rb'))


    distances = np.zeros(shape = (adjacency_matrix.shape[0]+1,adjacency_matrix.shape[0]+1))
    distances[1:,1:] = 100*adjacency_matrix.copy()
    from getToursAndPaths import getTour, readTourFile, getPathFromPrm, getFinalPath

#     getTour(distances, 'currTSP') # We just have an arbitrary name since it doesn't matter - can change this so that user can input filename if desired
    tour = readTourFile(tour_file, used_points)
    tour = (np.array(tour[1:])-1).tolist()
    indices = np.array(range(sampling_places.shape[0]))
    used_indices = indices[points_mask]
    tour_indices = used_indices[tour]

    # We then calculate the total distance travelled - and compute final trajectory:
    total_distance = 0 
    final_trajectory = []
    for i in range(len(tour)-1):
        current_point = tour[i]
        next_point = tour[i+1]
        total_distance += adjacency_matrix[current_point,next_point]
        nodes_path = nx.algorithms.shortest_path(roadmap,source = current_point,target = next_point,weight = 'weight')
    #     print(nodes_path)
    #     print(nodes_path)
        traj = [node_coords[j][:4] for j in nodes_path]
    #     print(trajectory)
        final_trajectory.append(traj[:-1])
    #     final_trajectory.append(pathDict[current_point,next_point])
    print("Total Distance Travelled by the EE = {} | Total Time Spent in Transit = {} ".format(total_distance,total_distance/30))






    #prepping animations
    time_allowances = np.linspace(0.00,0.5,50)
    times = solutions[0]
    partial_times  = []
    for allowance in time_allowances:
        partial_time = np.zeros(sampling_places.shape[0])

        for k in range(len(tour_indices)-1):
            total_time = times[tour_indices[:k+1]].sum()/(3600)
            if(total_time >= allowance):
                k = k-1
                break
        partial_time[tour_indices[:k+1]] = times[tour_indices[:k+1]]
        print(partial_time.sum()/3600)
        partial_times.append(partial_time)

    # trajectory_times:

    total_times = []
    for k in range(len(tour_indices)):
        partial_time = np.zeros(sampling_places.shape[0])
        partial_time[tour_indices[:k+1]] = times[tour_indices[:k+1]]
        total_times.append(partial_time.sum()/3600)
    #     print(total_times[-1])
        partial_times.append(partial_time)
    total_times = np.array(total_times)

    # we then determine the strategic stops
    first = np.argmin(np.abs(total_times-0.125))
    second = np.argmin(np.abs(total_times-0.25))
    third =  np.argmin(np.abs(total_times-0.375))
    fourth = total_times.shape[0] - 1



    stop_spots = [first,second,third,fourth]
    timed_trajectory = []
    trajectory_times = []
    prev_time = 0
    for index,path in enumerate(final_trajectory):
        section_length = len(path)
        section_times = np.zeros(section_length) + prev_time
        section_times = section_times + np.arange(section_length)*((total_times[index]-prev_time)/section_length)
        section_times = section_times*3600
        prev_time = total_times[index]
        timed_trajectory.extend(path)
        trajectory_times.extend(section_times)

    # setting up basic animations


    vis.init('PyQt')
    # del robot
    a = Geometry3D()
    res = a.loadFile(mesh_file)

    b = Geometry3D()
    res = b.loadFile(pcd_file)
    if(not res):
        print('failed loading geometry!')
    w = WorldModel()

    res = w.loadElement(robot_file)
    print(res)
#     w,robot,lamp,collider = setup_robot_and_light(mesh_file = mesh_file)
    robot = w.robot(0)
    cfig = robot.getConfig()
    cfig[2] = 0.08
    robot.setConfig(cfig)
    w.makeRigidObject("a")
    w.rigidObject(0).geometry().set(a)
    w.makeRigidObject("b")
    w.rigidObject(1).geometry().set(b)
    robot.link(3).appearance().setColor(210/255,128/255,240/255,1)

    robotTraj = RobotTrajectory(robot,milestones = timed_trajectory,times = trajectory_times).discretize(dt)
    linkTraj = robotTraj.getLinkTrajectory(linknum,dt) #.getPositionTrajectory([0,0,0])

    times = solutions[0]
    final_times = []
    for this_time in robotTraj.times:
        for j in range(len(tour_indices)):
            partial_time = np.zeros(sampling_places.shape[0])
            partial_time[tour_indices[:j+1]] = times[tour_indices[:j+1]]
            total_time = partial_time.sum()
    #         print(this_time,total_time)
            if(total_time >= this_time):
                break
        partial_time[tour_indices[j]] = this_time-times[tour_indices[:j]].sum()
        final_times.append(partial_time)
    final_times = np.array(final_times)
    # robot,w
    value = None
    cmap = None
    feature = None
#     time.sleep(20*np.random.rand())
    vis.add("A",a)
    # vis.add("B",b,pointSize = 15)
    vis.add("robot",robot)
    curr_milestones = sampling_places[tour_indices[:1],:]
    full_trajectory = []
    for i in final_trajectory:
        full_trajectory.extend(i)
    traj = trajectory.Trajectory(milestones= full_trajectory)#curr_milestones.tolist())
    # vis.add("traj",traj)
    # we then calculate the percentage of disinfection fluence:
    # disinfected_vecs = []
    partial_times_array = np.array(partial_times).transpose()
    # for times in partial_times:
    # mu = 80*np.matmul(visible_irradiance.transpose().toarray(),final_ties)
    # disinfected_vecs = np.clip(mu/(1.1*280),0,1).transpose()

    # we then add all zeros to begin the illustration
    # tmp = [np.zeros(disinfected_vecs[0].shape)]
    # tmp[0][0] = 1
    # disinfected_vecs_new = disinfected_vecs.copy()
    vis.visualization.setBackgroundColor(1, 1, 1, a=1)

    vis.show()
    
    # eliminating draw distance
    vis.lock()
    # time.sleep(0.5)
    vp = vis.getViewport()
    # vp.h = 640
    # vp.w = 640
    vp.clippingplanes = [0.1,10000]
    tform = pickle.load(open('tform.p','rb'))
    vp.setTransform(tform)
    vp.w = 1853//scale
    vp.h = 1123//scale
    # vis.setViewport(vp)
    vis.scene().setViewport(vp)
    vis.unlock()
#     time.sleep(5)
    new_partial_times = final_times
    max_t = 0
    for i in partial_times:
        max_t = max(max_t,np.max(i))

        
        #animation steps
    tholds = [0.25,0.5,0.75,0.999995]
    thold_index = 0
    thold = tholds[thold_index]
    # i = 56
    # while vis.shown():
    done = get_done_set(movie_folder)
    total_frames = set(range(len(robotTraj.milestones)))
    to_do = total_frames - done
#     pdb.set_trace()
    sc = Screenshotter(movie_dir)
    time.sleep(5)
    for i in sorted(list(to_do))[skip::max_step]:
    #     time.sleep((1/120)*solutions[0][tour_indices[i]])
    #     i = i%(len(disinfected_vecs_new))
        times = new_partial_times[i]
        disinfection_fractions = areas.copy()
        disinfection_fractions[:] = 0
        mu = 80*np.matmul(visible_irradiance.transpose().toarray(),times.transpose())
        disinfected_vecs = np.clip(mu/(1.1*280),0,1).transpose()
        if(i == 0 ):
            disinfected_vecs[0] = 100
#         print(disinfected_vecs.sum())
        #     disinfected_vecs[0] = 0.0001
    #     if(times.sum()/total_time >= thold):
    #         print('\nstopping at {} % due to total disinfection time {}\n'.format(thold,times.sum()/total_time))
    #         thold_index = (thold_index+1)%4
    #         thold = tholds[thold_index]
    #     if(i == 57):
    #         break
    #     time.sleep(0.2)
        fraction = areas[visible_points][disinfected_vecs >= 1].sum()/250.2
        
    #     traj = trajectory.Trajectory(milestones=so_far)

        disinfection_fractions[visible_points] = disinfected_vecs



        start = time.time()
        
        a_app = colorize.colorize(w.rigidObject(0),disinfection_fractions,'RdYlGn','faces',lighting=[0,0,-1])
        end = time.time()
    #     b_app = colorize.colorize(w.rigidObject(1),times,'magma')
    #     b_app.appearance().setPointSize(10)
    #     curr_pos = sampling_places[tour_indices[i],:]
        vis.lock()
        traj = Trajectory(milestones = linkTraj.getPositionTrajectory([0,0,0]).milestones[:i])

#         if(i != 0):
#         vis.remove("A")
        vis.add("A",a_app)
        
        if(i!= 0 ):
            vis.add("traj",traj)
        vis.addText('disinfection_progress','Reachable Coverage = {:.1f} %'.format(100*fraction),position = [10//scale,10//scale],size = 36//scale)
        vis.addText('disinfection_time','Time = {:.2f} minutes'.format(times.sum()/60),position = [10//scale,50//scale],size = 36//scale)
        robot.setConfig(robotTraj.milestones[i])

    #     vis.setAttribute("B","size",10)
    #     vis.remove("B")
    #     vis.add("B",b_app)
    #     vis.dirty("B")
    #     vis.remove("traj")

        vis.unlock()
#         time.sleep(1)
        sc.event.clear()
        sc.i = i
        vis.threadCall(sc.take_screenshot)
        sc.event.wait()
#         vis.threadCall(lambda :vis.scene().program.save_screen(movie_dir.format(i)))
    #     vis.scene().program.save_screen('{}.png'.format(i),multithreaded = False)
        print('\n {} took {}\n '.format(i,end-start))
    #     break
        time.sleep(0.2)
        

    #     i +=1
    #     time.sleep(5)
    vis.kill()
    
    
class Screenshotter:
    def __init__(self,movie_dir):
        self.event = Event()
        self.movie_dir = movie_dir
        self.i = 0
        self.event.clear()
    def take_screenshot(self):
        vis.scene().program.save_screen(self.movie_dir.format(self.i))
        self.event.set()    
    
def get_done_set(movie_dir):
    from glob import glob
    import pandas as pd
    a = glob(movie_dir + '/*.png')
#     print(a)
    if(a):
        a = pd.Series(a)
        b = a.str.split('/',expand = True).iloc[:,-1]
        c = b.str.split('.', expand = True).iloc[:,0].astype(int)
        d = set(c)
        return d
    else:
        return set({})

In [2]:
from joblib import Parallel,delayed
# generate_video(skip = 1,max_step = 1,dt = 0.5,scale = 1)
max_step = 5
# generate_video(skip = 0,max_step = 1,scale = 1)
Parallel(n_jobs = max_step,prefer = 'processes')(delayed(generate_video)(skip = skip, max_step = max_step,scale = 1) for skip in range(0,max_step))

log4cxx: Could not open file [./log4cxx.xml].
log4cxx: Could not open file [./log4cxx.xml].
log4cxx: Could not open file [./log4cxx.xml].
log4cxx: Could not open file [./log4cxx.xml].
log4cxx: Could not open file [./log4cxx.xml].


KrisLibrary::logger(): configured as default
LoadAssimp: Loaded model with 21224 verts and 34429 tris 
RobotWorld::LoadRobot: ./data/towerbot.rob
KrisLibrary::logger(): Logger RobParser has no appenders, using root logger.
Reading robot file ./data/towerbot.rob... 
   Parsing robot file, 4 links read... 
LoadAssimp: Loaded model with 66 verts and 128 tris 
LoadAssimp: Loaded model with 66 verts and 128 tris 
Loaded geometries in time 0.000355s, 256 total primitive elements 
Done loading robot file ./data/towerbot.rob 
KrisLibrary::logger(): configured as default
LoadAssimp: Loaded model with 21224 verts and 34429 tris 
RobotWorld::LoadRobot: ./data/towerbot.rob
KrisLibrary::logger(): Logger RobParser has no appenders, using root logger.
Reading robot file ./data/towerbot.rob... 
   Parsing robot file, 4 links read... 
LoadAssimp: Loaded model with 66 verts and 128 tris 
LoadAssimp: Loaded model with 66 verts and 128 tris 
Loaded geometries in time 0.000265s, 256 total primitive element

TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with 

TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with 

TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with 

...

TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 


 3929 took 0.2816288471221924
 
Saving screen to ./media/movies/towerbot/30_min/003934.png

 3934 took 0.3130021095275879
 
Saving screen to ./media/movies/towerbot/30_min/003939.png

 3939 took 0.23727035522460938
 
Saving screen to ./media/movies/towerbot/30_min/003944.png

 3944 took 0.3708336353302002
 
Saving screen to ./media/movies/towerbot/30_min/003949.png

 3949 took 0.2696387767791748
 
Saving screen to ./media/movies/towerbot/30_min/003954.png

 3954 took 0.3747882843017578
 
Saving screen to ./media/movies/towerbot/30_min/003959.png

 3959 took 0.23671197891235352
 
Saving screen to ./media/movies/towerbot/30_min/003964.png

 3964 took 0.25057435035705566
 
S

Saving screen to ./media/movies/towerbot/30_min/004374.pngTriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 


 3925 took 0.3196258544921875
 
Saving screen to ./media/movies/towerbot/30_min/003930.png

 3930 took 0.2329857349395752
 
Saving screen to ./media/movies/towerbot/30_min/003935.png

 3935 took 0.23510074615478516
 
Saving screen to ./media/movies/towerbot/30_min/003940.png

 3940 took 0.28287744522094727
 
Saving screen to ./media/movies/towerbot/30_min/003945.png

 3945 took 0.2959294319152832
 
Saving screen to ./media/movies/towerbot/30_min/003950.png

 

Saving screen to ./media/movies/towerbot/30_min/004371.pngTriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh 

TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with duplicate neighbors! 
  Triangle range 45 to 31345 
  May see strange results for some triangle mesh operations 
TriMeshTopology: mesh has 2298 triangles with 

[None, None, None, None, None]

In [ ]:
#
%load_ext wurlitzer


import pickle
import trimesh as tm
import pyrender
import numpy as np
import visibility
from importlib import reload
reload(visibility)
from visibility import Visibility, Mesh
from joblib import Parallel,delayed
from tqdm import tqdm
from scipy.sparse import lil_matrix
from klampt import WorldModel,Geometry3D
from klampt import vis
from klampt.math import so3,se3
import trimesh
import os
import random
import numpy as np
import math
from klampt.model.create import moving_base_robot
from klampt.io import resource
from klampt.math import vectorops,so3
from klampt.model.collide import WorldCollider
from klampt.model import ik
import time
import numpy as np
from tqdm import tqdm
from klampt import IKObjective
import networkx as nx
from klampt.model.trajectory import RobotTrajectory,Trajectory
from klampt.model.create import *
from klampt import vis
from klampt.robotsim import Geometry3D,WorldModel
from klampt.math import se3,so3,vectorops
from klampt.vis import colorize
import sys
import time
from klampt.model import trajectory



mesh_file = './full_detail_hospital_cad_meters_fixed.obj'
pcd_file = "towerbot_robot_used_points_hospital.pcd"
reachable_points_file = 'towerbot_reachable.p'
irradiance_file = 'towerbot_irradiance_matrix_35_divs.p'
sampling_places_file = 'towerbot_sampling_places_35_divs.p'
configs_file = 'configs.p'
robot_file = "./primitives/towerbot.rob"
solutions_file = 'towerbot_solutions.p'
adjacency_file = 'towerbot_adj.p'
roadmap_file = 'towerbot_roadmap.p'
node_coords_file = 'towerbot_node_coords.p'
movie_dir = '../media/movies/towerbot/30_min/{:06d}.png'
tour_file = 'Towerbot_currTSP.txt'
skip = 1
dt = 10
max_step = 20
scale = 1
linknum = 2

def get_solution_trajectory(mesh_file = './full_detail_hospital_cad_meters.obj',
                   pcd_file = "full_detail_robot_used_points_hospital.pcd",
                   reachable_points_file = 'full_detail_reachable.p',
                   irradiance_file = 'robot_clean_irradiance_matrix_35_divs.p',
                   sampling_places_file = 'armbot_sampling_places_35_divs.p',
                   configs_file = 'configs.p',
                   robot_file = "./primitives/disinfection_robot.rob",
                   solutions_file = 'robot_solutions.p',
                   adjacency_file = 'adj.p',
                   roadmap_file = 'roadmap.p',
                   node_coords_file = 'node_coords.p',
                   movie_dir = '../media/movies/armbot/30_min/{:06d}.png',
                   tour_file = 'currTSP.txt',
                   skip = 1,
                   dt = 0.4,
                   max_step = 20,
                   scale = 4,
                   linknum = 11,
                    power = 80):


    irradiance_matrix = pickle.load(open(irradiance_file,'rb'))
    reachable = pickle.load(open(reachable_points_file,'rb'))
    #     configs = pickle.load(open(configs_file,'rb'))
    sampling_places = pickle.load(open(sampling_places_file,'rb'))


    #     sampling_places = sampling_places[reachable]
    #     configs = np.array(configs)[reachable].tolist()
    mu_single = 60*60*80*0.5*irradiance_matrix
    print("\n\n number of reachable points : {} \n".format(np.sum(reachable)))




    #process irradiation spots
    irradiance_matrix = pickle.load(open(irradiance_file,'rb'))

    m=Mesh(mesh_file)
    areas = m.area()
    visible_points = np.asarray(((irradiance_matrix.sum(axis = 0)) > 0)).flatten()
    visible_areas = areas[visible_points]

    total_visible_areas = visible_areas.sum()
    total_visible_areas


    irradiance_matrix = irradiance_matrix[reachable]

    visible_points = np.asarray(((irradiance_matrix.sum(axis = 0)) > 0)).flatten()

    visible_irradiance = irradiance_matrix[:,visible_points]

    m=Mesh(mesh_file)
    areas = m.area()
    visible_areas = areas[visible_points]
    # we then define the visible area weights:



    visible_area_weights = np.exp(100*(visible_areas/visible_areas.sum()))
    visible_area_weights[:] = 1


    solutions = pickle.load(open(solutions_file,'rb'))
    visible_points = np.asarray(((irradiance_matrix.sum(axis = 0)) > 0)).flatten()
    visible_irradiance = irradiance_matrix[:,visible_points]



    # we now calculate the euclidean distance matrices for all the non-zero points:
    times = solutions[0]
    total_time = times.sum()
    points_mask = times>0.5
    used_points = sampling_places[points_mask,:]
    print(used_points.shape)
    mus = power*np.array(np.matmul(visible_irradiance.transpose(),times)).flatten()
    coverage = visible_areas[mus > 280].sum()/324.3
    print('Total Environment Coverage = {:.2f}%'.format(100*coverage))

    adjacency_matrix = pickle.load(open(adjacency_file,'rb'))
    roadmap = pickle.load(open(roadmap_file,'rb'))
    node_coords = pickle.load(open(node_coords_file,'rb'))

    from getToursAndPaths import getTour, readTourFile, getPathFromPrm, getFinalPath

    distances = np.zeros(shape = (adjacency_matrix.shape[0]+1,adjacency_matrix.shape[0]+1))
    distances[1:,1:] = 100*adjacency_matrix.copy()
    #     getTour(distances, 'currTSP') # We just have an arbitrary name since it doesn't matter - can change this so that user can input filename if desired
    tour = readTourFile(tour_file, used_points)
    tour = (np.array(tour[1:])-1).tolist()
    indices = np.array(range(sampling_places.shape[0]))
    used_indices = indices[points_mask]
    tour_indices = used_indices[tour]

    # We then calculate the total distance travelled - and compute final trajectory:
    total_distance = 0 
    final_trajectory = []
    for i in range(len(tour)-1):
        current_point = tour[i]
        next_point = tour[i+1]
        total_distance += adjacency_matrix[current_point,next_point]
        nodes_path = nx.algorithms.shortest_path(roadmap,source = current_point,target = next_point,weight = 'weight')
    #     print(nodes_path)
    #     print(nodes_path)
        traj = [node_coords[j][:4] for j in nodes_path]
    #     print(trajectory)
        final_trajectory.append(traj[:-1])
    # we then add the final point to the trajectory
    final_trajectory.append([traj[-1]])
    #     final_trajectory.append(pathDict[current_point,next_point])
    print("Total Distance Travelled by the EE = {} | Total Time Spent in Transit = {} ".format(total_distance,total_distance/30))






    #prepping animations
    time_allowances = np.linspace(0.00,0.5,50)
    times = solutions[0]
    partial_times  = []
    for allowance in time_allowances:
        partial_time = np.zeros(sampling_places.shape[0])

        for k in range(len(tour_indices)-1):
            total_time = times[tour_indices[:k+1]].sum()/(3600)
            if(total_time >= allowance):
                k = k-1
                break
        partial_time[tour_indices[:k+1]] = times[tour_indices[:k+1]]
        print(partial_time.sum()/3600)
        partial_times.append(partial_time)

    # trajectory_times:

    total_times = []
    for k in range(len(tour_indices)):
        partial_time = np.zeros(sampling_places.shape[0])
        partial_time[tour_indices[:k+1]] = times[tour_indices[:k+1]]
        total_times.append(partial_time.sum()/3600)
    #     print(total_times[-1])
        partial_times.append(partial_time)
    total_times = np.array(total_times)

    # we then determine the strategic stops
    first = np.argmin(np.abs(total_times-0.125))
    second = np.argmin(np.abs(total_times-0.25))
    third =  np.argmin(np.abs(total_times-0.375))
    fourth = total_times.shape[0] - 1



    stop_spots = [first,second,third,fourth]
    timed_trajectory = []
    trajectory_times = []
    prev_time = 0
    for index,path in enumerate(final_trajectory):
        section_length = len(path)
        section_times = np.zeros(section_length) + prev_time
        section_times = section_times + np.arange(section_length)*((total_times[index]-prev_time)/section_length)
        section_times = section_times*3600
        prev_time = total_times[index]
        timed_trajectory.extend(path)
        trajectory_times.extend(section_times)

    # setting up basic animations


    vis.init('PyQt')
    # del robot
    a = Geometry3D()
    res = a.loadFile(mesh_file)

    b = Geometry3D()
    res = b.loadFile(pcd_file)
    if(not res):
        print('failed loading geometry!')
    w = WorldModel()

    w.loadElement(robot_file)
    # w,robot,lamp,collider = setup_robot_and_light(mesh_file = mesh_file)
    robot = w.robot(0)
    robot.link(3).appearance().setColor(127/255,0/255,255/255,1)

    cfig = robot.getConfig()
    cfig[2] = 0.08
    robot.setConfig(cfig)
    w.makeRigidObject("a")
    w.rigidObject(0).geometry().set(a)
    w.makeRigidObject("b")
    w.rigidObject(1).geometry().set(b)

    final_timed_trajectory = np.zeros(shape = (len(timed_trajectory),4))
    final_timed_trajectory[:,2] = 0.08
    final_timed_trajectory[:,3] = 0
    final_timed_trajectory[:,:2] = np.array(timed_trajectory)


    robotTraj = RobotTrajectory(robot,milestones = final_timed_trajectory,times = trajectory_times).discretize(dt)
    linkTraj = robotTraj.getLinkTrajectory(linknum,dt) #.getPositionTrajectory([0,0,0])
    return linkTraj,robotTraj,final_timed_trajectory,robot
    times = solutions[0]
    final_times = []
    for this_time in robotTraj.times:
        for j in range(len(tour_indices)):
            partial_time = np.zeros(sampling_places.shape[0])
            partial_time[tour_indices[:j+1]] = times[tour_indices[:j+1]]
            total_time = partial_time.sum()
    #         print(this_time,total_time)
            if(total_time >= this_time):
                break
        partial_time[tour_indices[j]] = this_time-times[tour_indices[:j]].sum()
        final_times.append(partial_time)
    final_times[-1] = solutions[0]
    final_times = np.array(final_times)
    # robot,w
    value = None
    cmap = None
    feature = None
    #     time.sleep(20*np.random.rand())
    vis.add("A",a)
    # vis.add("B",b,pointSize = 15)
    vis.add("robot",robot)
    curr_milestones = sampling_places[tour_indices[:1],:]
    full_trajectory = []
    for i in final_trajectory:
        full_trajectory.extend(i)
    traj = trajectory.Trajectory(milestones= full_trajectory)#curr_milestones.tolist())
    # vis.add("traj",traj)
    # we then calculate the percentage of disinfection fluence:
    # disinfected_vecs = []
    partial_times_array = np.array(partial_times).transpose()
    # for times in partial_times:
    # mu = 80*np.matmul(visible_irradiance.transpose().toarray(),final_ties)
    # disinfected_vecs = np.clip(mu/(1.1*280),0,1).transpose()

    # we then add all zeros to begin the illustration
    # tmp = [np.zeros(disinfected_vecs[0].shape)]
    # tmp[0][0] = 1
    # disinfected_vecs_new = disinfected_vecs.copy()
    vis.visualization.setBackgroundColor(1, 1, 1, a=1)

    vis.show()

    # eliminating draw distance
    vis.lock()
    # time.sleep(0.5)
    vp = vis.getViewport()
    # vp.h = 640
    # vp.w = 640
    vp.clippingplanes = [0.1,10000]
    tform = pickle.load(open('tform.p','rb'))
    vp.setTransform(tform)
    vp.w = 1853//scale
    vp.h = 1123//scale

    # vis.setViewport(vp)
    vis.scene().setViewport(vp)
    vis.unlock()
    new_partial_times = final_times
    max_t = 0
    for i in partial_times:
        max_t = max(max_t,np.max(i))


        #animation steps
    tholds = [0.25,0.5,0.75,0.999995]
    thold_index = 0
    thold = tholds[thold_index]
    # i = 56
    # while vis.shown():
    for i in range(1508,len(robotTraj.milestones)):

    #     time.sleep((1/120)*solutions[0][tour_indices[i]])
    #     i = i%(len(disinfected_vecs_new))
        times = new_partial_times[i]
        disinfection_fractions = areas.copy()
        disinfection_fractions[:] = 0
        mu = 80*np.matmul(visible_irradiance.transpose(),times.transpose())
        disinfected_vecs = np.clip(mu/(1.1*280),0,1).transpose()
    #         print(disinfected_vecs.sum())
        #     disinfected_vecs[0] = 0.0001
    #     if(times.sum()/total_time >= thold):
    #         print('\nstopping at {} % due to total disinfection time {}\n'.format(thold,times.sum()/total_time))
    #         thold_index = (thold_index+1)%4
    #         thold = tholds[thold_index]
    #     if(i == 57):
    #         break
    #     time.sleep(0.2)
        fraction = areas[visible_points][disinfected_vecs >= 1].sum()/272
        vis.addText('disinfection_progress','Reachable Coverage = {:.1f} %'.format(100*fraction),position = [10//scale,10//scale],size = 36//scale)
        vis.addText('disinfection_time','Time = {:.2f} minutes'.format(times.sum()/60),position = [10//scale,50//scale],size = 36//scale)

    #     traj = trajectory.Trajectory(milestones=so_far)

        disinfection_fractions[visible_points] = disinfected_vecs




        a_app = colorize.colorize(w.rigidObject(0),disinfection_fractions,'RdYlGn','faces',lighting=[0,0,-1])
    #     b_app = colorize.colorize(w.rigidObject(1),times,'magma')
    #     b_app.appearance().setPointSize(10)
    #     curr_pos = sampling_places[tour_indices[i],:]
        vis.lock()
        robot.setConfig(robotTraj.milestones[i])
        if(i != 0):
            vis.remove("A")
            vis.add("A",a_app)
    #     vis.setAttribute("B","size",10)
    #     vis.remove("B")
    #     vis.add("B",b_app)
    #     vis.dirty("B")
    #     vis.remove("traj")
        traj = Trajectory(milestones = linkTraj.getPositionTrajectory([0,0,0]).milestones[:i])
        if(i!= 0 ):
            vis.add("traj",traj)
        vis.unlock()
        vis.threadCall(lambda :vis.scene().program.save_screen(movie_dir.format(i)))
    #     vis.scene().program.save_screen('{}.png'.format(i),multithreaded = False)
        print('\n {} \n '.format(i))
    #     break
    #         time.sleep(0.2)


    #     i +=1
    #     time.sleep(5)
    vis.kill()

In [ ]:
l,r,f,robot = get_solution_trajectory(mesh_file = './full_detail_hospital_cad_meters_fixed.obj',
               pcd_file = "towerbot_robot_used_points_hospital.pcd",
               reachable_points_file = 'towerbot_reachable.p',
               irradiance_file = 'towerbot_irradiance_matrix_35_divs.p',
               sampling_places_file = 'towerbot_sampling_places_35_divs.p',
               configs_file = 'configs.p',
               robot_file = "./primitives/towerbot.rob",
               solutions_file = 'towerbot_solutions.p',
               adjacency_file = 'towerbot_adj.p',
               roadmap_file = 'towerbot_roadmap.p',
               node_coords_file = 'towerbot_node_coords.p',
               movie_dir = '../media/movies/towerbot/30_min/{:06d}.png',
               tour_file = 'Towerbot_currTSP.txt',
               linknum = 2,scale = 1,power = 52)

In [ ]:
f

tj = trajectory.Trajectory(milestones= f, times = 1800*np.array(list(range(len(f))))/len(f))
# robotTraj = RobotTrajectory(robot,milestones = f)
# linkTraj = robotTraj.getLinkTrajectory(2,dt)
tj2  = tj.discretize(dt = 1)
len(tj2.milestones)

In [ ]:
milestones = np.array(tj2.milestones)
milestones = milestones[:,:2]

In [ ]:
milestones

In [ ]:
np.save('./towerbot_milestones.npy',milestones)

In [ ]:
from klampt import Geometry3D

In [ ]:
help(Geometry3D)